
<p align="center">
    <img src="mosef.png" alt="MOSEF Logo" width="300">
</p>
```
</p>

<a href="https://linkedin.com/in/lucasvazelle">Vazelle Lucas  </a> $\cdot$ 
<a href="https://www.linkedin.com/in/mariam-tarverdian">Taverdian Mariam </a> $\cdot$ 2024-2025, MOSEF $\cdot$ 


---


- [Projet Kaggel ](https://www.kaggle.com/competitions/mosef-bank-churn-prediction/)


il faut définir une stratégie de validation ! 

In [18]:
import pandas as pd


# 0. Import data <a class="anchor" id="Introduction"></a>
---


In [19]:
import pandas as pd

train_data_path = 'data/train.csv'
test_data_path = 'data/test.csv'

train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)

print("Training Data:")
display(train.head())

print("Testing Data:")
display(test.head())

Training Data:


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15713774.0,Hsieh,683.0,Germany,Male,49.0,7.0,123837.86,2.0,1.0,1.0,138700.75,1.0
1,1,15682533.0,Hs?eh,608.0,France,Female,30.0,10.0,0.00,2.0,1.0,1.0,146723.72,0.0
2,2,15791550.0,Tsou,539.0,France,Female,32.0,8.0,0.00,1.0,1.0,1.0,94873.60,0.0
3,3,15762762.0,Chiawuotu,530.0,Spain,Female,39.0,8.0,0.00,2.0,0.0,0.0,177896.93,0.0
4,4,15756475.0,Nnamdi,613.0,France,Female,45.0,10.0,0.00,2.0,1.0,1.0,116748.14,0.0


Testing Data:


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15000,15797736.0,Chikwendu,591.0,Germany,Female,32.0,2.0,106881.50,2.0,1.0,0.0,122810.53
1,15001,15676515.0,Lai,553.0,Spain,Male,37.0,2.0,0.00,2.0,1.0,0.0,63736.17
2,15002,15701291.0,Folliero,646.0,France,Female,29.0,7.0,0.00,2.0,1.0,0.0,121866.87
3,15003,15594812.0,Trevisani,603.0,Germany,Male,49.0,8.0,131394.56,1.0,0.0,1.0,147059.28
4,15004,15791321.0,Ch'ien,645.0,Spain,Female,39.0,3.0,0.00,2.0,1.0,0.0,129975.94


# 1. Premier traitement <a class="anchor" id="Introduction"></a>
---


j'aimerai que dans ce notbook on rajoute que des choses qui ont amélioré le modèle (test dans nos notbook individuels)


1. enlever variables d'informations inutiles 

2. gérer muticolinéarité 

3. encoder variable Gender Age (one hot encoding ou autre) 

4. Outlier

5. créer des variables pertinentes (min max,..)

6. Si plusieurs observtion correspondent à la même personne et qu'il prends un gros poids, tout sera biaisé..il faut vérifier l'indépendances des observations

7. garder que colonnes que l'on comprends : RFE, BORUTA,..

8. CHoix métrique

In [20]:
train_traitement = train.drop_duplicates()
train_traitement.count()

id                 15000
CustomerId         15000
Surname            15000
CreditScore        15000
Geography          15000
Gender             15000
Age                15000
Tenure             15000
Balance            15000
NumOfProducts      15000
HasCrCard          15000
IsActiveMember     15000
EstimatedSalary    15000
Exited             15000
dtype: int64

In [21]:
train_traitement = train_traitement.drop(columns=['Surname', 'id', 'CustomerId'])

In [22]:
train_traitement = pd.get_dummies(train_traitement, columns=['Gender', 'Geography'], drop_first=True)*1


# 2. Premier modèle <a class="anchor" id="Introduction"></a>
---


In [23]:
import statsmodels.api as sm

X = train_traitement.drop(columns=['Exited'])
y = train_traitement['Exited']

# Add a constant to the model
X = sm.add_constant(X)

# Fit the logistic regression model
logit_model = sm.Logit(y, X).fit()

# Print the summary of the model
print(logit_model.summary())



Optimization terminated successfully.
         Current function value: 0.315214
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 Exited   No. Observations:                15000
Model:                          Logit   Df Residuals:                    14988
Method:                           MLE   Df Model:                           11
Date:                Mon, 04 Nov 2024   Pseudo R-squ.:                  0.3689
Time:                        12:55:00   Log-Likelihood:                -4728.2
converged:                       True   LL-Null:                       -7492.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -3.2011      0.305    -10.499      0.000      -3.799      -2.604
CreditSc

In [24]:

test_traitement = test.drop(columns=['Surname', 'id', 'CustomerId'])
test_traitement = pd.get_dummies(test_traitement, columns=['Gender', 'Geography'], drop_first=True)*1
test_traitement = sm.add_constant(test_traitement)


In [25]:

test_predictions = logit_model.predict(test_traitement)
print(test_predictions)


0       0.227452
1       0.080427
2       0.047658
3       0.561553
4       0.210796
          ...   
9995    0.124726
9996    0.784692
9997    0.037106
9998    0.107152
9999    0.374166
Length: 10000, dtype: float64


# 4. Export prédiction <a class="anchor" id="Introduction"></a>
---


In [26]:
# Create a new dataframe with 'id' and 'test_predictions'
test_predictions_df = pd.DataFrame({'id': test['id'], 'Predictions': test_predictions})

# Display the new dataframe
print(test_predictions_df.head())

      id  Predictions
0  15000     0.227452
1  15001     0.080427
2  15002     0.047658
3  15003     0.561553
4  15004     0.210796


In [27]:
# Save the test_predictions_df to a CSV file
test_predictions_df.to_csv('test_predictions.csv', index=False)

lasso, ridge ,etc.

pycarest pour tester tous les modèles. 
LIGHT GBM


hyperpara 